<a href="https://colab.research.google.com/github/Tamiris499/Calculadora-/blob/main/tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Tratamento YouTube") \
    .getOrCreate()


In [2]:
df_video = spark.read.csv(
    "videos-stats.csv",
    header=True,
    inferSchema=True
)


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/videos-stats.csv. SQLSTATE: 42K03

In [ ]:
from pyspark.sql.functions import col

df_video = df_video.fillna(
    {"Likes": 0, "Comments": 0, "Views": 0}
)


In [ ]:
df_comentario = spark.read.csv(
    "comments.csv",
    header=True,
    inferSchema=True
)


In [ ]:
print("df_video:", df_video.count())
print("df_comentario:", df_comentario.count())


In [2]:
df_video = df_video.filter(col("Video ID").isNotNull())
df_comentario = df_comentario.filter(col("Video ID").isNotNull())

print("df_video sem nulos:", df_video.count())
print("df_comentario sem nulos:", df_comentario.count())

In [ ]:
df_video = df_video.dropDuplicates(["Video ID"])


In [ ]:
df_video = df_video \
    .withColumn("Likes", col("Likes").cast("int")) \
    .withColumn("Comments", col("Comments").cast("int")) \
    .withColumn("Views", col("Views").cast("int"))


In [ ]:
df_comentario = df_comentario \
    .withColumn("Likes", col("Likes").cast("int")) \
    .withColumn("Sentiment", col("Sentiment").cast("int")) \
    .withColumnRenamed("Likes", "Likes Comment")


In [ ]:
from pyspark.sql.functions import expr

df_video = df_video.withColumn(
    "Interaction",
    expr("Likes + Comments + Views")
)


In [ ]:
from pyspark.sql.functions import to_date

df_video = df_video.withColumn(
    "Published At",
    to_date(col("Published At"))
)


In [ ]:
from pyspark.sql.functions import year

df_video = df_video.withColumn(
    "Year",
    year(col("Published At"))
)


In [ ]:
df_join_video_comments = df_video.join(
    df_comentario,
    on="Video ID",
    how="left"
)


In [ ]:
df_us_videos = spark.read.csv(
    "USvideos.csv",
    header=True,
    inferSchema=True
)


In [ ]:
df_join_video_usvideos = df_video.join(
    df_us_videos,
    on="Title",
    how="left"
)

df_join_video_usvideos.show(5)


In [ ]:
from pyspark.sql.functions import sum as spark_sum

df_video.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in df_video.columns
]).show()


In [ ]:
df_video = df_video.drop("_c0")

df_video.write.mode("overwrite") \
    .parquet("videos-tratados-parquet")


In [ ]:
df_join_video_comments = df_join_video_comments.drop("_c0")

df_join_video_comments.write.mode("overwrite") \
    .parquet("videos-comments-tratados-parquet")
